In [3]:
!pip install --quiet sentence-transformers faiss-cpu PyPDF2 transformers


Install Packages

In [1]:
!pip install --upgrade pip
!pip install langchain==0.0.301
!pip install faiss-cpu
!pip install sentence-transformers
!pip install PyPDF2
!pip install transformers


Step 2 – Imports

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline


Step 3 – Load PDF and extract text

In [15]:
pdf_path = "/content/sample.pdf"  # Replace with your PDF path
reader = PdfReader(pdf_path)

all_text = ""
for page in reader.pages:
    all_text += page.extract_text() + " "


Step 4 – Split text into chunks

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
texts = text_splitter.split_text(all_text)


Step 5 – Create embeddings and FAISS vectorstore

In [17]:
# Use sentence-transformers embedding
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vectorstore
vectorstore = FAISS.from_texts(texts, embedding_model)


Step 6 – Load HuggingFace instruction model

In [19]:
pip install torch

In [21]:
import torch  #

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # GPU if available
)

from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cuda:0


Step 7 – Create RAG QA chain

In [22]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=False  # Set True if you want source chunks
)


Step 8 – Ask multiple questions

In [25]:
print("Type 'exit' to stop.\n")

while True:
    query = input("❓ Your Question: ")
    if query.lower() in ["exit", "quit"]:
        break
    answer = qa.run(query)
    print("\n📄 ANSWER:\n")
    print(answer)
    print("-" * 80)


Type 'exit' to stop.

❓ Your Question: what is AI

📄 ANSWER:

Artificial Intelligence (AI) is the field of computer science that focuses on creating systems capable of performing tasks that normally require human intelligence. Examples include reasoning, learning, problem -solving, perception, and language understanding.
--------------------------------------------------------------------------------
❓ Your Question: what is narrow intelligence

📄 ANSWER:

AI systems designed to perform a specific task, like voice assistants (Siri, Alexa) or recommendation systems. recommendation systems.
--------------------------------------------------------------------------------
❓ Your Question: what is General intelligence


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



📄 ANSWER:

AI that can perform a ny intellectual task that a human can do.
--------------------------------------------------------------------------------
❓ Your Question: what is super intelligence

📄 ANSWER:

An advanced form of AI that surpasses human intelligence
--------------------------------------------------------------------------------
❓ Your Question: exit
